In [151]:
!pip install scrapy

In [152]:
import time
import requests
import numpy as np
import pandas as pd
from scrapy.http import TextResponse
import re

# **Books to scrape** 



In [153]:
def booksto_scrape(url,base_url="http://books.toscrape.com/"):
    page = requests.get(url)
    response = TextResponse(body=page.text,url=url,encoding="utf-8")

    book_title=response.css("h3>a::attr(title)").extract()
    book_rating = response.css("p[class^='star-rating']::attr(class)").extract()
    instock  =response.css("p.price_color ~ p.instock::attr(class)").extract()
    instock_or_not = [i.replace("availability", " ") for i in instock]
    p_price = response.css("p.price_color::text").extract()
    price = [i.replace("Â", "") for i in p_price]
    book_page_URL = response.css("h3 >a::attr(href)").extract()
    book_picture_URL = response.css("img::attr(src)").extract()
    base_url = "http://books.toscrape.com/catalogue/"
    book_page_URL2 = [base_url + i for i in book_page_URL ]
    book_picture_URL2 = [base_url + i for i in book_picture_URL]
    rating = []
    for i in book_rating:
        rating.append(i.replace("star-rating", ""))
    book_genre = []
    book_description = []
    
    for i in book_page_URL2:
        page = requests.get(i)
        response = TextResponse(body=page.text,url=i,encoding="utf-8")

        book_genre.append(response.css("li~li~li > a::text")[0].extract())
        book_description.append(response.css("article[class='product_page'] > p::text").extract_first())
    
    return pd.DataFrame({ "price":price,"book_title":book_title, "rating":rating,"instock_or_not":instock_or_not,"book_genre":book_genre,"book_description":book_description,"book_page_URL2":book_page_URL2,"book_picture_URL2":book_picture_URL2}) 
    
    

In [154]:
bookslist = []
for i in range(1,1000):
    current_page =booksto_scrape(url = f"http://books.toscrape.com/catalogue/page-{i}.html")
    if current_page.shape[0] == 0:
        break
    else:
        bookslist.append(current_page)
bookslist = pd.concat(bookslist)
bookslist

,price,book_title,rating,instock_or_not,book_genre,book_description,book_page_URL2,book_picture_URL2
0,£51.77,A Light in the Attic,Three,instock,Poetry,It's hard to imagine a world without A Light i...,http://books.toscrape.com/catalogue/a-light-in...,http://books.toscrape.com/catalogue/../media/c...
1,£53.74,Tipping the Velvet,One,instock,Historical Fiction,"""Erotic and absorbing...Written with starling ...",http://books.toscrape.com/catalogue/tipping-th...,http://books.toscrape.com/catalogue/../media/c...
2,£50.10,Soumission,One,instock,Fiction,"Dans une France assez proche de la nÃ´tre, un ...",http://books.toscrape.com/catalogue/soumission...,http://books.toscrape.com/catalogue/../media/c...
3,£47.82,Sharp Objects,Four,instock,Mystery,"WICKED above her hipbone, GIRL across her hear...",http://books.toscrape.com/catalogue/sharp-obje...,http://books.toscrape.com/catalogue/../media/c...
4,£54.23,Sapiens: A Brief History of Humankind,Five,instock,History,From a renowned historian comes a groundbreaki...,http://books.toscrape.com/catalogue/sapiens-a-...,http://books.toscrape.com/catalogue/../media/c...
...,...,...,...,...,...,...,...,...
15,£55.53,Alice in Wonderland (Alice's Adventures in Won...,One,instock,Classics,None,http://books.toscrape.com/catalogue/alice-in-w...,http://books.toscrape.com/catalogue/../media/c...
16,£57.06,"Ajin: Demi-Human, Volume 1 (Ajin: Demi-Human #1)",Four,instock,Sequential Art,High school student Kei Nagai is struck dead i...,http://books.toscrape.com/catalogue/ajin-demi-...,http://books.toscrape.com/catalogue/../media/c...
17,£16.97,A Spy's Devotion (The Regency Spies of London #1),Five,instock,Historical Fiction,"In Englandâs Regency era, manners and elegan...",http://books.toscrape.com/catalogue/a-spys-dev...,http://books.toscrape.com/catalogue/../media/c...
18,£53.98,1st to Die (Women's Murder Club #1),One,instock,Mystery,"James Patterson, bestselling author of the Ale...",http://books.toscrape.com/catalogue/1st-to-die...,http://books.toscrape.com/catalogue/../media/c...


In [155]:
bookslist.to_excel('bookslist.xlsx', index=False)

In [156]:
data=(bookslist["price"])

In [157]:
type(data)

pandas.core.series.Series

In [158]:
bookslist[bookslist["price"] == bookslist["price"].max()]["book_genre"]

8    Romance
Name: book_genre, dtype: object

The expensive genre is Romance

In [159]:
bookslist["price"].max()

'£59.99'

In [160]:
bookslist["rating"].value_counts()

 One      226
 Three    203
 Two      196
 Five     196
 Four     179
Name: rating, dtype: int64

226 books are one star
203 books are three star and so on